### Rondon's head_motion_prediction ingest and exploration

In [ ]:
import os
import sys

project_path = f"predict360user/head_motion_prediction/"
sys.path.append(project_path)
from predict360user.head_motion_prediction.David_MMSys_18.Read_Dataset import (
    load_sampled_dataset,
    plot_3d_trace,
)

# show one traject
ONE_USER = "0"
ONE_VIDEO = "10_Cows"
if not "SAMPLE_DATASET" in locals():
    print(f"get SAMPLE_DATASET from {project_path}")
    cwd = os.getcwd()
    os.chdir(project_path)  # load_sampled_datasetuses relative path
    SAMPLE_DATASET = load_sampled_dataset()
    os.chdir(cwd)

traces = SAMPLE_DATASET[ONE_USER][ONE_VIDEO][:, 1:]

# plot 3d
plot_3d_trace(traces, ONE_USER, ONE_VIDEO)

### predict360user: Ingest and exploration

In [ ]:
import predict360user as p3u

In [ ]:
df_trajecs = p3u.load_df_trajecs(dataset="david")
print(f"number of trajecs in total {df_trajecs.size}")
print("\nnumber of trajecs per df_trajecs:")
print(df_trajecs.reset_index()["df_trajecs"].value_counts())
print("\nnumber of trajecs per entropy class:")
print(df_trajecs["actS_c"].value_counts())
print("\ndf head")
display(df_trajecs.head())

In [ ]:
# show one traject
p3u.show_traject(df_trajecs.loc["david", "0", "10_Cows"])

In [ ]:
# entropy distribution
p3u.show_entropy_histogram(df_trajecs)

In [ ]:
# show representive trajectory for each entropy class
p3u.show_trajects_representative(df_trajecs)

In [ ]:
p3u.split(df_trajecs)
# entropy distribution per partition
p3u.show_entropy_histogram_per_partition(df_trajecs)

### predict360user.Train

In [ ]:
import predict360user as p3u

In [ ]:
# load dataset and split
df = p3u.load_df_wins(dataset="david")
df = p3u.split(df)

# fit model
cfg = p3u.RunConfig()
model = p3u.get_model(cfg)
model.fit(df)

# evaluate and log to wandb
err_per_class_dict = model.evaluate(model, df)